### Just example code from the quickstart guide. I did find it helpful to read through this real quick so here's the link if you're interested.
https://huggingface.co/docs/setfit/quickstart

In [1]:
pip install setfit

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install transformers==4.45.2
# This will fix AttributeError: 'CallbackHandler' object has
# no attribute 'tokenizer if you're running into that issue
# Try 4.42.2 if that doesn't work

In [4]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

c:\Users\deth_\miniconda3\envs\setfit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [5]:
from datasets import load_dataset

dataset = load_dataset("SetFit/sst2")
dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 872
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
})

In [6]:
from setfit import sample_dataset

train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=8)
train_dataset

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 16
})

In [7]:
test_dataset = dataset["test"]
test_dataset

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 1821
})

In [8]:
model.labels = ["negative", "positive"]

In [9]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=32,
    num_epochs=10,
)

In [10]:
from setfit import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

c:\Users\deth_\miniconda3\envs\setfit\lib\site-packages\codecarbon\input.py:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
c:\Users\deth_\miniconda3\envs\setfit\lib\site-packages\pkg_resources\__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
Map: 100%|██████████| 16/16 [00:00<00:00, 3196.57 examples/s]


In [11]:
trainer.train()

***** Running training *****
  Num unique pairs = 144
  Batch size = 32
  Num epochs = 10
  2%|▏         | 1/50 [00:01<01:32,  1.88s/it]

{'embedding_loss': 0.2383, 'grad_norm': 0.6014610528945923, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.2}


100%|██████████| 50/50 [01:02<00:00,  1.18s/it]

{'embedding_loss': 0.1, 'grad_norm': 0.3201802670955658, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 50/50 [01:03<00:00,  1.18s/it]c:\Users\deth_\miniconda3\envs\setfit\lib\site-packages\codecarbon\output_methods\file.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
100%|██████████| 50/50 [01:03<00:00,  1.28s/it]

{'train_runtime': 63.7667, 'train_samples_per_second': 22.582, 'train_steps_per_second': 0.784, 'train_loss': 0.10276843756437301, 'epoch': 10.0}


In [ ]:
model.save_pretrained("setfit-bge-small-v1.5-sst2-8-shot") #  Saves a local mode to be loaded later

In [13]:
model = SetFitModel.from_pretrained("setfit-bge-small-v1.5-sst2-8-shot")
# how to load the model above

In [14]:
preds = model.predict([
    "It's a charming and often affecting journey.",
    "It's slow -- very, very slow.",
    "A sometimes tedious film.",
])
preds

['positive', 'negative', 'negative']